In [ ]:
import math
from pathlib import Path

import ipywidgets as widgets
import pandas as pd
import seaborn.objects as so
from itables import init_notebook_mode

init_notebook_mode(all_interactive=True)

In [ ]:
logs = [f for f in Path("results").iterdir() if f.is_file()]

data = []
for log in logs:
    stream = open(log)
    while line := stream.readline():
        words = line.split()
        if words[1] == "detected":
            timestamp = words[0]
            level = words[3]
            data.append([log.name, "detection", timestamp, level])
        elif words[1] == "debug":
            timestamp = words[0]
            idx = int(words[3])
            sample = int(words[5])

            w = [int(x) for x in stream.readline().split()[1:]]
            s = [int(x) for x in stream.readline().split()[1:]]
            decomposition_level = list(range(len(s)))
            data.append(
                [
                    log.name,
                    "debug",
                    timestamp,
                    math.nan,
                    idx,
                    sample,
                    decomposition_level,
                    w,
                    s,
                ]
            )
        else:
            print("bug in parsing", file=sys.stderr)

In [ ]:
df = pd.DataFrame(
    data,
    columns=[
        "capture",
        "type",
        "timestamp",
        "detected_level",
        "index",
        "sample",
        "decomposition_level",
        "w",
        "s",
    ],
)

In [ ]:
df = df.explode(["decomposition_level", "w", "s"], ignore_index=True)

df["capture"] = df["capture"].astype("category")
df["type"] = df["type"].astype("category")
df["timestamp"] = pd.to_datetime(df["timestamp"])
df["detected_level"] = df["detected_level"].astype("Int64")
df["decomposition_level"] = df["decomposition_level"].astype("Int64")
df["index"] = df["index"].astype("Int64")
df["sample"] = df["sample"].astype("Int64")
df["w"] = df["w"].astype("Int64")
df["s"] = df["s"].astype("Int64")
df

In [ ]:
captures = df["capture"].unique()
timestamps = {}
for capture in captures:
    timestamps[capture] = df[df["capture"] == capture]["timestamp"]

initial_capture = captures[0]
initial_timestamps = df[df["capture"] == initial_capture]["timestamp"]

capture_dropdown = widgets.Dropdown(options=captures, description="Capture:")

timestamp_slider = widgets.SelectionSlider(
    options=initial_timestamps, description="Timestamp", layout={"width": "800px"}
)


def update_timestamp_options(change):
    selected_capture = change["new"]
    timestamp_slider.options = timestamps[selected_capture]


capture_dropdown.observe(update_timestamp_options, names="value")


def update_plot(capture, timestamp):
    filtered_data = df[(df["capture"] == capture) & (df["timestamp"] == timestamp)]

    p = so.Plot(filtered_data, "decomposition_level", "s").add(so.Line()).add(so.Dots())
    display(p)


widgets.interactive(update_plot, capture=capture_dropdown, timestamp=timestamp_slider)